In [ ]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# .env 파일에서 환경변수 로드
load_dotenv(dotenv_path='../.env')

# 환경변수에서 API 키 불러오기
OpenAI.api_key = os.getenv("OPENAI_API_KEY")

# 프롬프트 구성 요소별 변수로 관리
persona = """
당신은 중소형 IT 업체의 실무 담당자입니다. 5년 정도의 금융권 프로젝트 경험이 있으며, 주로 유지보수와 소규모 개선 작업을 담당해왔습니다. 제안서 작성 경험이 많지 않아 RFP 요구사항을 그대로 따르는 것에 집중합니다. 화려한 기술이나 높은 목표보다는 확실하게 달성 가능한 범위 내에서 안전하게 제안하는 스타일입니다.
"""

concept = """
이번 제안은 '최소 요구사항 충족형' 컨셉입니다. RFP에서 요구하는 필수 사항만을 정확히 이행하는 데 집중하며, 추가적인 제안이나 혁신적인 기술은 최소화합니다. 검증된 기본 기술 스택을 사용하고, 보수적인 성능 지표를 제시하여 리스크를 최소화합니다. 간단명료하게 작성하되, 필수 항목은 빠뜨리지 않습니다.
"""

goal_context = """
당신은 지금 {수협은행의 데이터기반 실시간 개인화 마케팅 시스템 구축} 사업의 입찰 경쟁에 참여하고 있습니다.
해당 사업의 추진 목표는 고객 데이터를 활용하여 대면·비대면 통합 옴니채널 기반의 상시적 고객관리 활동 지원 인프라 구축입니다.
<해당 사업의 추진 배경 및 목적>
##중심 고객기반으로 세대 교체에 따른 마케팅 인프라 고도화
  ###비대면 마케팅 자동화를 통한 업무 효율화 및 마케팅 정교화
##온·오프라인 채널간 유기적 연계를 통한 마케팅 품질 향상
  ###다양한 접점의 고객 데이터 활용 비대면 마케팅 실행력 확보
##데이터 통합 분석을 통한 고객 맞춤 관리프로그램 실행
  ###섬세한 타겟팅 및 세분화된 고객군별 마케팅 전략 수립
</해당 사업의 추진 배경 및 목적>
<주요 개발 내용>
- 실시간 수집 서버 : 인터넷뱅킹, 모바일뱅킹, 모바일웹, 거래데이터, 상담 데이터 등 실시간 데이터 수집, 정제, 가공 및 모니터링
- 캠페인 시스템 (배치형 캠페인/ 실시간 캠페인) - 캠페인 설계 : 일회성, 주기성, 실시간 캠페인에 대한 설계 정보 등록 및 관리
	- 캠페인 실행 : 설계된 캠페인 정보기반으로 캠페인 자동 실행 (대상자추출 > 중복제거 > 채널 발송) 
	- 캠페인 분석 : 캠페인 모니터링 및 채널 반응, 성과 분석 등에 대한 분석 정보 제공
	- 캠페인 관리 : 캠페인 설계, 실행에 필요한 기본 정보 관리
- 캠페인 DB 구축
	- 캠페인 마트 DB : 캠페인 대상 선정 및 성과측정에 필요한 고객(거래· 행동·상담)데이터, 반응 및 성과데이터 DB구축
	- 마케팅 운영 DB : 마케팅 운영에 필요한 캠페인 및 대상고객 정보, 이력정보 구성
- 데이터 및 시스템 연계 : 발송 채널 및 유관 시스템과 연계
</주요개발내용>

이 제안서의 목표는 우리의 기본적인 기술력과 성실한 수행 의지를 보여주어 사업자로 선정되는 것입니다.
"""

instructions = """
위의 페르소나, 제안서 컨셉, 목표에 완벽하게 빙의하여, 아래의 요구사항을 충족하는 '입찰 제안서'를 Markdown 형식으로 작성하세요.

###중요: 금융권 최소 수준의 보수적인 수치 가이드라인
다음 범위 내에서 **안전하고 보수적인** 수치를 사용하세요:

**시스템 성능 지표**
- **서비스 가용률**: 95.0% ~ 96.0% (계획된 유지보수 및 예상 장애 포함)
- **평균 응답시간**: 3~5초
- **최대 응답시간**: 10~15초
- **동시 접속자 처리**: 500~800명
- **시간당 처리량**: 3만~5만 건
- **데이터베이스 쿼리 응답시간**: 300~500ms

**데이터 처리 지표**
- **배치 처리 시간**: 3~4시간 (일별 집계 기준)
- **실시간 이벤트 처리 지연**: 15~30초
- **데이터 정합성**: 95~97%

**마케팅 캠페인 성과 지표**
- **캠페인 발송 성공률**: 80~85%
- **이메일 오픈율**: 15~18%
- **SMS 도달률**: 90~92%
- **클릭율 (CTR)**: 3~4%
- **마케팅 효율 개선 목표**: 1차년도 15%, 2차년도 20%

**안정화 및 운영 방안**
- **시스템 안정화 기간**: 오픈 후 3~4개월
- **장애 등급별 대응체계**:
  - 긴급(Critical): 장애 인지 후 1시간 이내 대응 착수, 목표 복구시간(RTO) 8시간
  - 높음(High): 장애 인지 후 4시간 이내 대응 착수, RTO 1영업일
  - 보통(Medium): 업무시간 내 대응, RTO 2~3영업일
- **백업 정책**:
  - 전체 백업: 주 1회 (주말)
  - 증분 백업: 주 2~3회
  - RPO (목표 복구 시점): 최대 1일 이내
- **무상 유지보수**: 시스템 오픈 후 12개월

###제안서 작성 방식
1. **간단명료하게**: 각 섹션을 1~2단락으로 간결하게 작성
2. **필수 항목 위주**: RFP 필수 요구사항만 다룸
3. **검증된 기술**: 널리 사용되는 기본 기술 스택 (Java/Spring, JSP, Oracle/MySQL, 기본 REST API)
4. **보수적 표현**: "안정적", "검증된", "확실한" 등의 표현 사용

###요구사항
1. 아래의 제안서 구조를 따라 작성하되, 각 항목을 간단히 작성하세요.
<제안서 구조>
1. 제안 개요
    - 추진 배경
    - 목표 및 핵심 KPI
    - 범위
2. 제안 업체 일반 현황
    - 제안사 개요
    - 유사 프로젝트 수행 경험 (1~2개)
    - 재무 현황
3. 프로젝트 수행
    3.1 기술 부문
    - 시스템 구축 전략
    - 시스템 구성
    - 시스템 구축 방안
    - 테스트 방안
    - 보안 관리 방안
    
    3.2 일반 부문
    - 프로젝트 일정
    - 투입 인력 계획
    - 개발 방법론
4. 사업 관리
    4.1 사업 관리 방안
    4.2 품질 보증 계획
    4.3 유지보수 지원 방안
5. 결론
</제안서 구조>

2. 각 항목은 1~2단락으로 간단히 작성
3. 기술적 세부사항은 최소화
4. 플레이스홀더 없이 모든 섹션 작성
"""

# 최종 프롬프트 조합
prompt = f"{persona}\n{concept}\n{goal_context}\n{instructions}"

# 최신 openai 라이브러리 방식 (openai>=1.0.0)
client = OpenAI(api_key=OpenAI.api_key)
response = client.chat.completions.create(
    model="gpt-5-nano",
    messages=[
        {"role": "user", "content": prompt}
    ]
)

In [ ]:
# 답변 출력
try:
    print(response.choices[0].message.content)
except Exception as e:
    print(f"출력 에러: {e}")

In [ ]:
import re

def markdown_to_structured_html(markdown_text):
    """
    Markdown 텍스트를 구조화된 HTML로 변환
    """
    html_lines = ['<!DOCTYPE html>', '<html>', '<head>', 
                  '<meta charset="UTF-8">', 
                  '<title>수협은행 제안서 - 기본형</title>',
                  '<style>',
                  'body { font-family: "Malgun Gothic", sans-serif; line-height: 1.6; margin: 20px; max-width: 1200px; margin: 0 auto; padding: 20px; }',
                  'h1 { color: #2c3e50; border-bottom: 3px solid #3498db; padding-bottom: 10px; }',
                  'h2 { color: #34495e; margin-top: 30px; border-bottom: 2px solid #95a5a6; padding-bottom: 8px; }',
                  'h3 { color: #7f8c8d; margin-top: 20px; }',
                  'table { border-collapse: collapse; width: 100%; margin: 20px 0; }',
                  'th, td { border: 1px solid #ddd; padding: 12px; text-align: left; }',
                  'th { background-color: #3498db; color: white; }',
                  'ul, ol { margin: 10px 0; padding-left: 30px; }',
                  'p { margin: 10px 0; }',
                  'strong { color: #2c3e50; }',
                  '</style>',
                  '</head>', '<body>']
    
    lines = markdown_text.split('\n')
    in_table = False
    in_list = False
    
    for line in lines:
        line = line.strip()
        
        if not line:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<br/>')
            continue
        
        # 제목 변환
        if line.startswith('# '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h1>{line[2:]}</h1>')
        elif line.startswith('## '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h2>{line[3:]}</h2>')
        elif line.startswith('### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h3>{line[4:]}</h3>')
        elif line.startswith('#### '):
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append(f'<h4>{line[5:]}</h4>')
        
        # 표 감지
        elif '|' in line and not in_table:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            html_lines.append('<table>')
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<th>{cell}</th>')
            html_lines.append('</tr>')
            in_table = True
        elif '|' in line and in_table:
            if '---' in line:
                continue
            cells = [cell.strip() for cell in line.split('|') if cell.strip()]
            html_lines.append('<tr>')
            for cell in cells:
                html_lines.append(f'<td>{cell}</td>')
            html_lines.append('</tr>')
        elif in_table and '|' not in line:
            html_lines.append('</table>')
            in_table = False
        
        # 목록 변환
        elif line.startswith('- ') or line.startswith('* '):
            if not in_list:
                html_lines.append('<ul>')
                in_list = True
            content = line[2:].strip()
            content = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', content)
            html_lines.append(f'<li>{content}</li>')
        
        # 일반 텍스트
        else:
            if in_list:
                html_lines.append('</ul>')
                in_list = False
            line = re.sub(r'\*\*(.+?)\*\*', r'<strong>\1</strong>', line)
            html_lines.append(f'<p>{line}</p>')
    
    if in_table:
        html_lines.append('</table>')
    if in_list:
        html_lines.append('</ul>')
    
    html_lines.extend(['</body>', '</html>'])
    
    return '\n'.join(html_lines)

# 제안서를 구조화된 HTML로 변환
proposal_text = response.choices[0].message.content
structured_html = markdown_to_structured_html(proposal_text)

# HTML 파일로 저장
output_path = 'proposal_basic_3of5.html'
with open(output_path, 'w', encoding='utf-8') as f:
    f.write(structured_html)

print(f"구조화된 HTML 제안서가 생성되었습니다: {output_path}")
print(f"총 {len(structured_html)} 문자")
print(structured_html)